# Redes Neurais e sua Implementação

In [59]:
import string
import re
import nltk
import pandas as pd
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ruasgar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
df = pd.read_csv("../data/final_dataset.csv")

In [54]:
df.columns

Index(['sentence', 'news_link', 'outlet', 'topic', 'type', 'group_id',
       'num_sent', 'Label_bias', 'Label_opinion', 'article', 'biased_words4',
       'full_article'],
      dtype='object')

## 1 - Features Importantes
- Label_bias (categórica): Indica se o texto foi classificado como enviesado, não-enviesado ou se não foi possível atingir um consenso quanto a classificação.
- Label_opinion (categórica): Indica de que modo o viés se manifesta na percepção dos entrevistados; especificamente separando casos de exposição de opinião do autor ou com fatos que corroborem um viés. (Um pouco fuzzy demais, talvez?)
- Biased_words(vetor): Indica as palavras marcadas como "denunciantes" da presença de viés. 
- Topic(categórica): Indica o assunto do texto, dentro das categorias PREENCHER AQUI   

## 2 - Pré-Processamento

### 2.1 - Básico textual (remoção de stopwords, filtro de expressões com números, etc)

- A priori, vamos considerar como tokens todas as palavras que:
    - tenham mais do que 3 letras
    - não possuam números
    - não estejam nas stopwords do inglês
    
- Vamos remover os acentos e, mediante a escolha do usuário, aplicar um stemmer.
- Visando o mapeamento das palavras para números inteiros, vamos utilizar o padrão de marcar alguns tokens especiais:
    - 0: padding (completar o tamanho das representações)
    - 1: Start-of-Sequence
    - 2: tokens desconhecidos, necessários para novas palavras

In [55]:
digit_pattern = r'\d+(\.\d+)?'
solo_quotations_pattern = pattern = r"^(?:' '|\\" "|\`\`)$"

remove_quotation = r"'\b|\b'\s|\s'\b|\"\b|\b\"\s|\s\"\b|``\b|\b``\s|\s``\b" # como em "America is Great"
remove_symbols_only = r'\b[^\w\s]+\b' # como em '--'
remove_symbols_if_aside = r'\b[^\w\s]|[^\w\s]\b' # como em 'dog-'

remove_patterns = remove_quotation,remove_symbols_if_aside, remove_symbols_only

stop_words = set(stopwords.words('english')) # talvez o set deixe mais rápido

def preprocess_sentences(df):
    
    prepped_sentences = []

    for s in df['sentence']:
        # Remove acentos e põe tudo para lower case
        s = unidecode(s).lower()
        tokens = word_tokenize(s,language="english")

        tokens = [
            re.sub('|'.join(remove_patterns),'',t)
            for t in tokens 
            if not bool(re.search(digit_pattern,t))
            and t not in string.punctuation
            and t not in stop_words
        ]

        tokens = [
            t for t in tokens
            if not bool(re.match(solo_quotations_pattern,t)) 
            and len(t) >= 3
        ]
        
        prepped_sentences.append(tokens)

    df['sentence'] = prepped_sentences

    return df 

df_functions_test_copy = df.copy()
df_functions_test_copy = preprocess_sentences(df_functions_test_copy)
df_functions_test_copy['sentence'].head()


0    [youtube, making, clear, birtherism, platform,...
1    [increasingly, bitter, dispute, american, wome...
2    [may, humanitarian, crisis, driving, vulnerabl...
3    [professor, teaches, climate, change, classes,...
4    [world, antidoping, agency, tuesday, said, rus...
Name: sentence, dtype: object

### 2.1 - Stemmers

- Temos o objetivo de avaliar como diferentes estratégias de stemming afetam os resultados do treinamento. A função a seguir visa abstrair esse passo para um etapa separada de pré-processamento. 
- Serão testados os stemmer de Porter e de Lancaster, além da lematização via WordNet. Vamos também obter os resultados do modelo quando utilizados sem qualquer stemmer. 

In [56]:
def apply_stemmer(sentence, stemmer_type:str):

    # Função a ser executada para cada sentença, com expectativa de uso em algum filtro, por exemplo

    stemmer = {}
    lemmatizer = {}

    if stemmer_type == "Porter":
        stemmer = PorterStemmer()
        return [stemmer.stem(token) for token in sentence]

    elif stemmer_type == "Lancaster":
        stemmer = LancasterStemmer()
        return [stemmer.stem(token) for token in sentence]
    
    else:  # "Wordnet":
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.stem(token) for token in sentence]


- Aplicando stemmer para teste da função 

In [57]:
df_functions_test_copy['sentence'] = df_functions_test_copy['sentence'].apply(lambda x: apply_stemmer(x,"Porter"))
df_functions_test_copy['sentence'].head()

0    [youtub, make, clear, birther, platform, year,...
1    [increasingli, bitter, disput, american, women...
2    [may, humanitarian, crisi, drive, vulner, peop...
3    [professor, teach, climat, chang, class, subje...
4    [world, antidop, agenc, tuesday, said, russian...
Name: sentence, dtype: object

### 1.3 - Encoding do Vocabulário

### 2.4 - Separando o dataset

In [63]:
def train_valid_test_split(features, labels):
    """ Retorna uma lista de tuplas contendo os datasets de features e de labels para cada segmento (treino, validação, teste) """
    
    # Treino-val e Teste
    shuffle_train_test = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=892)
    train_val_indexes, test_indexes = next(shuffle_train_test.split(features.values, labels.values))
    train_val_df, train_val_labels = features.iloc[train_val_indexes], labels.iloc[train_val_indexes]
    test_df, test_labels = features.iloc[test_indexes], labels.iloc[test_indexes]

    # Treino e Validação
    shuffle_train_validate = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=124)
    train_indexes, validation_indexes = next(shuffle_train_validate.split(train_val_df.values,train_val_labels.values))
    train_df, train_labels = features.iloc[train_indexes], labels.iloc[train_indexes]
    validation_df, validation_labels = features.iloc[validation_indexes], labels.iloc[validation_indexes]


    return [(train_df, train_labels), (validation_df, validation_labels), (test_df, test_labels)]


In [71]:
train, val, test = train_valid_test_split(df_functions_test_copy,df_functions_test_copy['type'])
train[0].shape

(848, 12)